In [0]:
import requests
import unicodedata
import re
from bs4 import BeautifulSoup

In [0]:
#crawler sul sito web www.paginainizio.com (sezione ricette)

max_p = 24878

recipes_dict = {}
recipes_hyper = {}
recipes_link = {}
recipes_no_duplicates = {}

check = "Bevanda a base di " 

#estrae i titoli delle ricette dalle pagine incrementando l'id ad ogni chiamata
#input: numero massimo di pagine da visitare

def trade_spider(max_pages):
    
    page = 1
    
    while page <= max_pages:
        
        flag = None
        s_page = str(page)
        
        while(len(s_page)<=8):
            s_page = "0" + s_page
        #print(s_page)
        
        url = "http://www.paginainizio.com/public/ricette/recipe.php?recipe=" + s_page
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
                
        for elem in soup.find_all('td'):
            if(check in elem):
                flag = True
        
        if not flag:
             for title in soup.find_all('title'):
                    if(title != None):
                        recipes_dict[page] = title.string 
                        
        page += 1
        print("PAGINA: ",page)
        

In [0]:
trade_spider(max_p)

In [0]:
#rimozione del testo racchiuso in parentesi tonde

for key in recipes_dict.keys():
    if(recipes_dict[key] != None):
        temp = recipes_dict[key]
        temp = re.sub(r'\([^)]*\)', '', temp)
        temp = temp.rstrip()
        recipes_dict[key] = temp

In [0]:
#rimozione dei termini superflui come 'Ricetta' e 'con foto'
#trasformazione stringhe in lowercase

for key in recipes_dict.keys():
    if(recipes_dict[key] != None):
        ric = recipes_dict[key].replace('Ricetta ', '')
        ric_1 = ric.replace(' con foto', '') 
        recipes_dict[key] = ric_1.lower()

In [0]:
#rimozione dei duplicati

for key, value in recipes_dict.items():
    if value not in recipes_no_duplicates.values():
        recipes_no_duplicates[key] = value
        print(recipes_no_duplicates[key])

In [0]:
print(len(recipes_no_duplicates.items()))

In [0]:
recipes = []

#rimozione caratteri speciali, punteggiatura

for key in recipes_no_duplicates.keys():
    if(recipes_no_duplicates[key] != None):
        temp = re.sub(u'[^a-zA-Z0-9àèùòìáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', '', recipes_no_duplicates[key])
        recipes_no_duplicates[key] = temp
        recipes.append(recipes_no_duplicates[key])

In [0]:
# salva il nome delle ricette in un file csv

csvFilename = "Dataset/RecipesNoDrink.csv" 
csv = open(csvFilename, "w") 
colNames = "Recipes\n" 
csv.write(colNames)

In [0]:
for elem in recipes:
    if(elem != None):
        csv.write(elem + "\n")